Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

**Description of the data**
```
/datasets/car_data.csv
Features:
    DateCrawled — date profile was downloaded from the database
    VehicleType — vehicle body type
    RegistrationYear — vehicle registration year
    Gearbox — gearbox type
    Power — power (hp)
    Model — vehicle model
    Mileage — mileage (measured in km due to dataset's regional specifics)
    RegistrationMonth — vehicle registration month
    FuelType — fuel type
    Brand — vehicle brand
    NotRepaired — vehicle repaired or not
    DateCreated — date of profile creation
    NumberOfPictures — number of vehicle pictures
    PostalCode — postal code of profile owner (user)
    LastSeen — date of the last activity of the user
Target:
    Price — price (Euro)
```

## Data preparation

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import math
import time

from category_encoders import BinaryEncoder, OrdinalEncoder

from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import make_scorer, mean_squared_error 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_colwidth', None) # no wrap
pd.set_option('display.max_columns', 100) # columns number
pd.set_option('display.max_rows', 50) # rows number

In [4]:
car_dates = ['DateCrawled', 'DateCreated', 'LastSeen']

try:
    df = pd.read_csv('./datasets/car_data.csv', parse_dates=car_dates)
except:
    df = pd.read_csv('/datasets/car_data.csv', parse_dates=car_dates)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Mileage            354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  NotRepaired        283215 non-null  object        
 12  DateCreated        354369 non-null  datetime64[ns]
 13  NumberOfPictures   354369 non-null  int64   

In [6]:
df.describe(include='all')

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354369,354369.000,316879,354369.000,334536,354369.000,334664,354369.000,354369.000,321474,354369,283215,354369,354369.000,354369.000,354369
unique,15470,NaN,8,NaN,2,NaN,250,NaN,NaN,7,40,2,109,NaN,NaN,18592
top,2016-05-03 14:25:00,NaN,sedan,NaN,manual,NaN,golf,NaN,NaN,petrol,volkswagen,no,2016-03-04 00:00:00,NaN,NaN,2016-07-04 07:16:00
freq,66,NaN,91457,NaN,268251,NaN,29232,NaN,NaN,216352,77013,247161,13719,NaN,NaN,654
first,2016-01-04 00:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-10-03 00:00:00,NaN,NaN,2016-01-04 00:15:00
last,2016-12-03 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-12-03 00:00:00,NaN,NaN,2016-12-03 23:54:00
mean,NaN,4416.657,NaN,2004.234,NaN,110.094,NaN,128211.173,5.715,NaN,NaN,NaN,NaN,0.000,50508.689,NaN
std,NaN,4514.159,NaN,90.228,NaN,189.850,NaN,37905.342,3.726,NaN,NaN,NaN,NaN,0.000,25783.096,NaN
min,NaN,0.000,NaN,1000.000,NaN,0.000,NaN,5000.000,0.000,NaN,NaN,NaN,NaN,0.000,1067.000,NaN
25%,NaN,1050.000,NaN,1999.000,NaN,69.000,NaN,125000.000,3.000,NaN,NaN,NaN,NaN,0.000,30165.000,NaN


In [7]:
report = df.isna().sum().to_frame().rename(columns = {0: 'missing_values'})
report = report[report['missing_values']>0]
report['% of total'] = (report['missing_values'] / df.shape[0] * 100).round(2)
report.sort_values(by = 'missing_values', ascending = False)

,missing_values,% of total
NotRepaired,71154,20.080
VehicleType,37490,10.580
FuelType,32895,9.280
Gearbox,19833,5.600
Model,19705,5.560


In [8]:
#Model is one of the main features, and filling it doesn't make any sense, so drop

df.dropna(subset=['Model'], inplace=True)

In [9]:
#we can't fill NotRepaired in any meaningful way, so unknown

df['NotRepaired'] = df['NotRepaired'].fillna('unknown')

In [10]:
#VehicleType, FuelType and Gearbox could be filled with mode based on Model and RegistrationYear

df['VehicleType'] = df['VehicleType'].fillna(df.groupby(['Model','RegistrationYear'])['VehicleType']\
                                             .transform(lambda x: next(iter(x.mode()), np.nan)))
df['VehicleType'] = df['VehicleType'].fillna(df.groupby(['Model'])['VehicleType']\
                                             .transform(lambda x: next(iter(x.mode()), np.nan)))

df['FuelType'] = df['FuelType'].fillna(df.groupby(['Model','RegistrationYear'])['FuelType']\
                                       .transform(lambda x: next(iter(x.mode()), np.nan)))
df['FuelType'] = df['FuelType'].fillna(df.groupby(['Model'])['FuelType']\
                                       .transform(lambda x: next(iter(x.mode()), np.nan)))

df['Gearbox'] = df['Gearbox'].fillna(df.groupby(['Model','RegistrationYear'])['Gearbox']\
                                     .transform(lambda x: next(iter(x.mode()), np.nan)))
df['Gearbox'] = df['Gearbox'].fillna(df.groupby(['Model'])['Gearbox']\
                                     .transform(lambda x: next(iter(x.mode()), np.nan)))

In [11]:
#there are some strange values in RegistrationYear we better get rid of

ageikr = df['RegistrationYear'].quantile(0.75) - df['RegistrationYear'].quantile(0.25)
age_toohigh = df['RegistrationYear'].quantile(0.75) + 1.5 * ageikr
age_toolow = df['RegistrationYear'].quantile(0.25) - 1.5 * ageikr

wrong_age = df[(df['RegistrationYear'] < age_toolow) | (df['RegistrationYear'] > age_toohigh)].shape[0]
print(f'Outliers in RegistrationYear: {wrong_age} ({(wrong_age / df.shape[0]):.2%})')

df = df[(df['RegistrationYear'] >= age_toolow) & (df['RegistrationYear'] <= age_toohigh)]

Outliers in RegistrationYear: 4939 (1.48%)


In [12]:
#there are some too large values in Power we better get rid of

powerikr = df['Power'].quantile(0.75) - df['Power'].quantile(0.25)
power_toohigh = df['Power'].quantile(0.75) + 1.5 * powerikr

wrong_power = df[(df['Power'] > power_toohigh)].shape[0]
print(f'Outliers in Power: {wrong_power} ({(wrong_power / df.shape[0]):.2%})')
df = df[df['Power'] <= age_toohigh]

Outliers in Power: 6753 (2.05%)


In [13]:
#separate RegistrationMonth feature doesn't make much sense. 
#let's construct a Registration from year and month combined

df['Registration'] = df['RegistrationYear'].astype(str) + '-' + df['RegistrationMonth'].astype(str)

In [14]:
print(f'Duplicates in car data: {df.duplicated().sum()} ({(df.duplicated().sum() / df.shape[0]):.2%})')

Duplicates in car data: 264 (0.08%)


In [15]:
df.drop_duplicates(keep=False, inplace=True, ignore_index=True)

In [16]:
df.drop(columns=['DateCrawled', 'RegistrationYear', 'RegistrationMonth', 'DateCreated', \
                 'NumberOfPictures', 'PostalCode', 'LastSeen'], inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329104 entries, 0 to 329103
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Price         329104 non-null  int64 
 1   VehicleType   329104 non-null  object
 2   Gearbox       329104 non-null  object
 3   Power         329104 non-null  int64 
 4   Model         329104 non-null  object
 5   Mileage       329104 non-null  int64 
 6   FuelType      329104 non-null  object
 7   Brand         329104 non-null  object
 8   NotRepaired   329104 non-null  object
 9   Registration  329104 non-null  object
dtypes: int64(3), object(7)
memory usage: 25.1+ MB


In [18]:
df.sample(10)

,Price,VehicleType,Gearbox,Power,Model,Mileage,FuelType,Brand,NotRepaired,Registration
223460,6900,convertible,manual,120,2_reihe,150000,petrol,peugeot,no,2007-3
234540,12500,suv,manual,140,touran,125000,gasoline,volkswagen,no,2008-0
9362,549,small,manual,75,other,150000,petrol,citroen,yes,1999-6
323409,5900,small,manual,87,corsa,60000,petrol,opel,no,2011-6
300471,17750,suv,auto,140,tiguan,125000,gasoline,volkswagen,no,2011-5
20930,700,sedan,manual,75,golf,150000,petrol,volkswagen,no,1998-10
224714,1900,wagon,manual,116,passat,150000,gasoline,volkswagen,no,1999-1
257194,800,wagon,auto,150,a4,150000,petrol,audi,yes,1998-10
202829,130,sedan,auto,0,e_klasse,5000,gasoline,mercedes_benz,unknown,2012-0
16144,19995,bus,auto,170,touran,80000,gasoline,volkswagen,no,2012-4


**Conclusion**  
There are some missing values in this dataset. Therefore, we dropped the entries with missing `Model`, filled in `NotRepaired` with 'unknown' and filled `VehicleType`, `FuelType` and `Gearbox` with a mode based on Model and RegistrationYear.

We dropped some outliers from `RegistrationYear` (1.48%) and `Power` (2.05%) features.

We had 0.08% of duplicates to drop.

Also, having a separate `RegistrationMonth` feature isn't very helpful, so we constructed a `Registration` feature from `RegistrationYear` and `RegistrationMonth` combined.

And finally, we dropped `DateCrawled`, `DateCreated`, `NumberOfPictures`, `PostalCode`, and `LastSeen` since they are not about the vehicle but about the advert and could not affect the price.

## Model training

In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('Price', axis=1), df['Price'], test_size=0.25, random_state=42
)

In [20]:
#making a dataset with categorical features for CatBoost and LGBM

X_train_cat = X_train.copy()
X_test_cat = X_test.copy()
for col in X_train_cat.select_dtypes(include=['object']):
    X_train_cat[col] = X_train_cat[col].astype('category')
for col in X_test_cat.select_dtypes(include=['object']):
    X_test_cat[col] = X_test_cat[col].astype('category')    
    
cat_features = X_train_cat.select_dtypes(include=['category']).columns.values    

In [21]:
def rmse_func(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

rmse = make_scorer(rmse_func, greater_is_better=False)

def fit_score(model, pref, name, params):
    start = time.time()
    if pref == 'lr':
        pipe = Pipeline([('encoder', BinaryEncoder()), ('scaler', MinMaxScaler()), (pref, model)])
        model = GridSearchCV(pipe, param_grid=params, cv=5, scoring=rmse, verbose=0, refit=True)
        model.fit(X_train, y_train)
        score = abs(model.best_score_)
        best_params = model.best_params_
    elif pref == 'rf':
        pipe = Pipeline([('encoder', OrdinalEncoder()), ('scaler', MinMaxScaler()), (pref, model)])
        model = GridSearchCV(pipe, param_grid=params, cv=5, scoring=rmse, verbose=0, refit=True)
        model.fit(X_train, y_train)
        score = abs(model.best_score_)
        best_params = model.best_params_
    elif pref == 'cb':
        grid_search_result = model.grid_search(params, X=Pool(X_train_cat, y_train, cat_features=cat_features))
        score = abs(model.get_best_score()['learn']['RMSE'])
        best_params = grid_search_result['params']
    else:
        pipe = Pipeline([(pref, model)])
        model = GridSearchCV(pipe, param_grid=params, cv=5, scoring=rmse, verbose=0, refit=True)
        model.fit(X_train_cat, y_train)
        score = abs(model.best_score_)
        best_params = model.best_params_
    end = time.time()
    diff = end - start
    return pd.DataFrame([[name, score, diff, best_params]], \
                        columns=['Estimator', 'Best Score', 'Time', 'Params'])


In [22]:
%%time
LinearRegressor = fit_score(LinearRegression(), 'lr', 'Linear Regressor', {})

CPU times: user 18.4 s, sys: 3.56 s, total: 22 s
Wall time: 17.1 s


In [23]:
%%time
RandomForest = fit_score(RandomForestRegressor(random_state=42), 'rf', 'Random Forest',
                       {'rf__n_estimators': [5, 10],
                        'rf__max_depth': [3, 5],
                        'rf__min_impurity_decrease': [0.05, 0.25, 0.1]})

CPU times: user 2min 11s, sys: 5.16 s, total: 2min 16s
Wall time: 2min 16s


In [24]:
%%time
CatBoost = fit_score(CatBoostRegressor(random_seed=42, silent=True, loss_function='RMSE'), 'cb', 'Cat Boost',
                       {'iterations': [50],
                        'depth': [10],
                        'learning_rate': [0.01, 0.05, 0.1],})


bestTest = 4467.028606
bestIteration = 49

0:	loss: 4467.0286060	best: 4467.0286060 (0)	total: 2.11s	remaining: 4.21s

bestTest = 2187.248425
bestIteration = 49

1:	loss: 2187.2484252	best: 2187.2484252 (1)	total: 4.36s	remaining: 2.18s

bestTest = 1889.202563
bestIteration = 49

2:	loss: 1889.2025630	best: 1889.2025630 (2)	total: 6.66s	remaining: 0us
Estimating final quality...
Training on fold [0/3]

bestTest = 1884.493571
bestIteration = 49

Training on fold [1/3]

bestTest = 1879.948526
bestIteration = 49

Training on fold [2/3]

bestTest = 1893.619184
bestIteration = 49

CPU times: user 1min 25s, sys: 5.11 s, total: 1min 30s
Wall time: 14.9 s


In [25]:
%%time
LGBM = fit_score(LGBMRegressor(random_seed = 42), 'lgbm', 'LGBM',
                       {'lgbm__max_depth':[3],
                        'lgbm__num_leaves' : [5, 11],
                        'lgbm__learning_rate': [0.1, 1],
                        'lgbm__n_estimators': [50]})

CPU times: user 57.4 s, sys: 5.9 s, total: 1min 3s
Wall time: 9.34 s


In [26]:
scores = pd.concat([LinearRegressor, RandomForest, CatBoost, LGBM]).reset_index(drop=True)
scores.style.hide_index()

Estimator,Best Score,Time,Params
Linear Regressor,3223.459721,17.047784,{}
Random Forest,2925.622575,136.169318,"{'rf__max_depth': 5, 'rf__min_impurity_decrease': 0.05, 'rf__n_estimators': 10}"
Cat Boost,1825.110312,14.910052,"{'depth': 10, 'iterations': 50, 'learning_rate': 0.1}"
LGBM,1952.282805,9.340586,"{'lgbm__learning_rate': 1, 'lgbm__max_depth': 3, 'lgbm__n_estimators': 50, 'lgbm__num_leaves': 11}"


**Conclusion**  
We tried to train several different models using the RMSE score and GridSearchCV to look for the best hyperparameters. The lower the RMSE, the better is the model, so our best is the CatBoost Regressor.

## Model analysis

In [27]:
final_model = CatBoostRegressor(random_seed=42, silent=True, loss_function='RMSE',\
                                **scores[scores['Estimator']=='Cat Boost']['Params'].values[0])
final_model.fit(X=Pool(X_train_cat, y_train, cat_features=cat_features))

cb_rmse = rmse_func(y_test, final_model.predict(X_test_cat))
print(f'RMSE score for best CatBoostRegressor model on test dataframe: {cb_rmse:.2f}')

RMSE score for best CatBoostRegressor model on test dataframe: 1858.45


**Conclusion** 
We explored several models using the RMSE score and GridSearchCV to look for the best hyperparameters. CatBoost Regressor shows not only the best score but is training faster than Random Forest or LGBM models.